# Preparar clasificación de noticias RPP
Este cuaderno:
1. Cargamos las 50 noticias desde `data/rpp_source.json`.
2. Generamos el bloque numerado para pedir clasificación a un LLM (0-World, 1-Sports, 2-Business, 3-Science/Tech).
3. Guardamos el resultado clasificado en `data/rpp_classified.json`.

In [1]:
# === Celda 0: sincronizar ruta del repo en Colab ===
import os, sys

In [2]:
# 1️. Borrar clonaciones anteriores
!rm -rf /content/News_Classification-lab_LovatonValeria_PizarroSebastian

# 2️. Clonar el repo completo desde GitHub
!git clone https://github.com/ValLovaton/News_Classification-lab_LovatonValeria_PizarroSebastian
%cd /content/News_Classification-lab_LovatonValeria_PizarroSebastian


# 3️⃣ Mostrar estructura
print("\n📂 Estructura inicial:")
!ls -R | head -40

# 4️⃣ Asegurar ruta base y carpeta data
sys.path.append(os.path.abspath("."))
os.makedirs("data", exist_ok=True)

print("\n✅ Ubicación actual:", os.getcwd())


Cloning into 'News_Classification-lab_LovatonValeria_PizarroSebastian'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 101 (delta 45), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 99.67 KiB | 5.54 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/News_Classification-lab_LovatonValeria_PizarroSebastian

📂 Estructura inicial:
.:
data
notebooks
outputs
README.md
requirements.txt
src

./data:
rpp_classified.json
rpp_source.json

./notebooks:
agnews_train_eval.ipynb
agnews_train_eval_step2_roberta.ipynb
agnews_train_eval_step3_compare.ipynb
bonus_prepare_rpp.ipynb
bonus_prepare_rpp.ipynbbonus_prepare_rpp.ipynb

./outputs:

./src:
train_eval.py

✅ Ubicación actual: /content/News_Classification-lab_LovatonValeria_PizarroSebastian


In [3]:
import json, pandas as pd, os

#1. Cargar las 50 noticias del RSS
with open('data/rpp_source.json', 'r', encoding='utf-8') as f:
    news = json.load(f)
df = pd.DataFrame(news)
print(f'Noticias cargadas: {len(df)}')
df.head(2)

Noticias cargadas: 50


,title,description,link,date_published
0,Ollanta Humala se pronuncia tras anulación de ...,"El expresidente Humala, sentenciado a 15 años ...",https://rpp.pe/politica/judiciales/ollanta-hum...,"Mon, 20 Oct 2025 22:19:10 -0500"
1,"Vecina de San Isidro considera que hubo ""tardí...","Cynthia Yamamoto, vocera de 'Peruanos de a pie...",https://rpp.pe/lima/actualidad/san-isidro-veci...,"Mon, 20 Oct 2025 22:01:43 -0500"


In [4]:
# === Reparar conflictos de versiones (seguro en Colab) ===
!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 -q
!pip install transformers accelerate -q
import torch, transformers
print("✅ Versiones sincronizadas:")
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)


✅ Versiones sincronizadas:
Torch: 2.8.0+cu126
Transformers: 4.57.1


In [5]:
from transformers import pipeline

candidate_labels = ["World", "Sports", "Business", "Science/Technology"]
label_to_id = { "World":0, "Sports":1, "Business":2, "Science/Technology":3 }

# Modelo multilingüe (zero-shot)
classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
)

preds = []
for it in news:
    text = f"{it['title']} — {it['description']}"
    result = classifier(text, candidate_labels=candidate_labels, multi_label=False)
    label = result["labels"][0]
    preds.append(label_to_id[label])

df["label_llm"] = preds
df.to_json("data/rpp_classified.json", orient="records", indent=2, force_ascii=False)
print("✅ Archivo guardado en data/rpp_classified.json con 50 etiquetas generadas por LLM (open-source).")
df.head(5)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ Archivo guardado en data/rpp_classified.json con 50 etiquetas generadas por LLM (open-source).


,title,description,link,date_published,label_llm
0,Ollanta Humala se pronuncia tras anulación de ...,"El expresidente Humala, sentenciado a 15 años ...",https://rpp.pe/politica/judiciales/ollanta-hum...,"Mon, 20 Oct 2025 22:19:10 -0500",0
1,"Vecina de San Isidro considera que hubo ""tardí...","Cynthia Yamamoto, vocera de 'Peruanos de a pie...",https://rpp.pe/lima/actualidad/san-isidro-veci...,"Mon, 20 Oct 2025 22:01:43 -0500",2
2,Riveros acabó en el piso: así fue superado el ...,Juan Lucumí de Ayacucho FC silenció el Estadio...,https://rpp.pe/futbol/descentralizado/ver-gol-...,"Mon, 20 Oct 2025 21:30:33 -0500",1
3,"Partidos de hoy, martes 21 de octubre del 2025...",Fútbol EN VIVO | Horarios y canales de TV para...,https://rpp.pe/futbol/futbol-mundial/partidos-...,"Mon, 20 Oct 2025 22:00:58 -0500",1
4,Lluvias en Huancayo: ¿qué daños provocaron en ...,"Los distritos de El Tambo, Chilca, Sapallanga ...",https://rpp.pe/peru/junin/lluvias-en-huancayo-...,"Mon, 20 Oct 2025 21:05:01 -0500",0


In [6]:
# === Celda 3: Comparar los modelos fine-tuned vs. las etiquetas del LLM ===
import os, glob, torch, seaborn as sns, matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score
import pandas as pd, numpy as np, json

# 1️⃣ Cargar las etiquetas creadas por el modelo open-source
with open("data/rpp_classified.json", "r", encoding="utf-8") as f:
    rpp = json.load(f)

texts = [f"{x['title']} — {x['description']}" for x in rpp]
y_llm = np.array([int(x["label_llm"]) for x in rpp])

# 2️⃣ Definir rutas de checkpoints de tus modelos entrenados (ajusta si usaste otras)
roots = {
    "roberta-base": "outputs/roberta-base",
    "microsoft/deberta-v3-base": "outputs/microsoft/deberta-v3-base",
    "bert-base-multilingual-cased": "outputs/bert-base-multilingual-cased"
}

def last_ckpt(root):
    """Busca el último checkpoint dentro de cada carpeta de modelo."""
    if not os.path.exists(root):
        return None
    cks = sorted(glob.glob(os.path.join(root, "checkpoint-*")))
    return cks[-1] if cks else root

# 3️⃣ Calcular F1 comparando predicciones del modelo vs LLM
f1_vs_llm = {}
for name, root in roots.items():
    ckpt = last_ckpt(root)
    if not ckpt or not os.path.exists(ckpt):
        print(f"⚠️ No encontré checkpoint para {name} en {root}, se omite.")
        continue
    try:
        tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=True)
        model = AutoModelForSequenceClassification.from_pretrained(ckpt).eval()
        enc = tokenizer(texts, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        with torch.no_grad():
            logits = model(**enc).logits
        y_pred = logits.argmax(dim=-1).cpu().numpy()
        f1 = f1_score(y_llm, y_pred, average="macro")
        f1_vs_llm[name] = f1
        print(f"✅ {name}: F1 vs LLM = {f1:.4f}")
    except Exception as e:
        print(f"⚠️ Error con {name}: {e}")

# 4️⃣ Visualización comparativa
if f1_vs_llm:
    df_llm = pd.DataFrame(list(f1_vs_llm.items()), columns=["Modelo", "F1_vs_LLM"])
    plt.figure(figsize=(8,5))
    sns.barplot(x="Modelo", y="F1_vs_LLM", data=df_llm, palette="crest")
    plt.title("Consistencia entre modelos y clasificaciones del LLM (RPP)", fontsize=13)
    plt.ylabel("F1-score (comparado con LLM)")
    plt.xlabel("Modelos fine-tuned")
    plt.ylim(0,1)
    plt.tight_layout()
    plt.show()
    display(df_llm)
else:
    print("⚠️ No se pudo evaluar ningún modelo (verifica rutas o checkpoints).")



⚠️ No encontré checkpoint para roberta-base en outputs/roberta-base, se omite.
⚠️ No encontré checkpoint para microsoft/deberta-v3-base en outputs/microsoft/deberta-v3-base, se omite.
⚠️ No encontré checkpoint para bert-base-multilingual-cased en outputs/bert-base-multilingual-cased, se omite.
⚠️ No se pudo evaluar ningún modelo (verifica rutas o checkpoints).
